In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Rheumatoid_Arthritis"
cohort = "GSE121894"

# Input paths
in_trait_dir = "../../input/GEO/Rheumatoid_Arthritis"
in_cohort_dir = "../../input/GEO/Rheumatoid_Arthritis/GSE121894"

# Output paths
out_data_file = "../../output/preprocess/Rheumatoid_Arthritis/GSE121894.csv"
out_gene_data_file = "../../output/preprocess/Rheumatoid_Arthritis/gene_data/GSE121894.csv"
out_clinical_data_file = "../../output/preprocess/Rheumatoid_Arthritis/clinical_data/GSE121894.csv"
json_path = "../../output/preprocess/Rheumatoid_Arthritis/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Gene expression profile of endothelial cells derived from circulating progenitors issued from patients with rheumatoid arthritis"
!Series_summary	"Synovial neoangiogenesis is an early and crucial event to promote the development of the hyperplasic proliferative pathologic synovium in rheumatoid arthritis (RA). Endothelial cells (ECs) are critical for the formation of new blood vessels since they highly contribute to angiogenesis and vasculogenesis."
!Series_summary	"To better characterize these cells, our group has studied the gene expression profiles of ECs issued from 18 RA patients compared to 11 healthy controls."
!Series_overall_design	"ECs derived from circulating endothelial progenitor cells (EPCs) were isolated from peripheral blood of RA patients and controls for RNA extraction and hybridization on Affymetrix microarrays. Gene expression profiles of EPC-derived ECs were determined in basal conditions and also after hypoxic exposure."
Samp

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
import os
from typing import Optional, Callable, Any, Dict
import json

# 1. Gene Expression Data Availability
# Based on the series title and summary, this appears to be gene expression data from microarrays
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
trait_row = 0  # 'subject status' indicates RA vs healthy control
age_row = None  # Age information is not available in the sample characteristics
gender_row = None  # Gender information is not available in the sample characteristics

# 2.2 Data Type Conversion
def convert_trait(value: str) -> int:
    """Convert trait value to binary (0 for control, 1 for RA)"""
    if value is None:
        return None
    
    # Extract value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    if 'rheumatoid arthritis' in value.lower():
        return 1  # RA patient
    elif 'healthy control' in value.lower():
        return 0  # Healthy control
    else:
        return None  # Unknown

def convert_age(value: str) -> Optional[float]:
    """Convert age value to continuous"""
    # Not used as age data is not available
    return None

def convert_gender(value: str) -> Optional[int]:
    """Convert gender value to binary (0 for female, 1 for male)"""
    # Not used as gender data is not available
    return None

# 3. Save Metadata
# Determine trait data availability
is_trait_available = trait_row is not None

# Conduct initial filtering and save relevant information
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Check if trait_row is not None, which means clinical data is available
if trait_row is not None:
    # Assuming clinical_data is a DataFrame from a previous step
    # Let's load the clinical data
    files = os.listdir(in_cohort_dir)
    clinical_file = None
    for file in files:
        if "characteristics" in file.lower():
            clinical_file = os.path.join(in_cohort_dir, file)
            break
    
    if clinical_file:
        clinical_data = pd.read_csv(clinical_file, sep='\t', header=0)
        
        # Extract clinical features
        selected_clinical_df = geo_select_clinical_features(
            clinical_df=clinical_data,
            trait=trait,
            trait_row=trait_row,
            convert_trait=convert_trait,
            age_row=age_row,
            convert_age=convert_age,
            gender_row=gender_row,
            convert_gender=convert_gender
        )
        
        # Preview the data
        preview = preview_df(selected_clinical_df)
        print(f"Clinical data preview: {preview}")
        
        # Create directory if it doesn't exist
        os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
        
        # Save the data
        selected_clinical_df.to_csv(out_clinical_data_file, index=False)
        print(f"Clinical data saved to {out_clinical_data_file}")


### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row IDs (gene or probe identifiers) for future observation.
print(gene_data.index[:20])


Index(['100009613_at', '100009676_at', '10000_at', '10001_at', '10002_at',
       '100033423_at', '100033424_at', '100033425_at', '100033426_at',
       '100033436_at', '100033444_at', '100033453_at', '100033806_at',
       '100033820_at', '100037417_at', '100038246_at', '10003_at',
       '100048912_at', '100049587_at', '100049716_at'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Based on my biomedical knowledge, these are not standard human gene symbols
# The "_at" suffix suggests these are likely probe IDs from a microarray platform (e.g., Affymetrix)
# Human gene symbols would typically be in formats like "BRCA1", "TP53", etc.
# These identifiers will need to be mapped to standard gene symbols for analysis

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['1_at', '10_at', '100_at', '1000_at', '10000_at'], 'ENTREZ_GENE_ID': ['1', '10', '100', '1000', '10000'], 'Description': ['alpha-1-B glycoprotein', 'N-acetyltransferase 2 (arylamine N-acetyltransferase)', 'adenosine deaminase', 'cadherin 2, type 1, N-cadherin (neuronal)', 'v-akt murine thymoma viral oncogene homolog 3'], 'SPOT_ID': [1.0, 10.0, 100.0, 1000.0, 10000.0]}


### Step 6: Gene Identifier Mapping

In [7]:
# 1. Let's see more of the gene annotation data to better understand the available columns
print("Gene annotation column names:", gene_annotation.columns.tolist())
print("\nFirst 5 rows of gene annotation data:")
print(gene_annotation.head(5))

# 2. Looking at this microarray data format (Affymetrix), the ENTREZ_GENE_ID is likely 
# the most reliable mapping to official gene symbols. Let's use it for mapping.
# However, we need a way to convert Entrez IDs to gene symbols.

# Since the function apply_gene_mapping will attempt to extract human gene symbols from text,
# we need a column that contains gene symbols or names from which symbols can be extracted.
# The Description column is our best option in this case.

# Extract the mapping columns
gene_mapping = get_gene_mapping(gene_annotation, 'ID', 'Description')

# 3. Apply the gene mapping to convert probe-level data to gene-level expression
gene_data = apply_gene_mapping(gene_data, gene_mapping)

# Normalize the gene symbols to ensure consistency
gene_data = normalize_gene_symbols_in_index(gene_data)

# Preview the mapped gene data
print("\nAfter normalization - First 10 gene symbols:")
print(gene_data.index[:10])
print("\nTotal number of genes after mapping:", len(gene_data))


Gene annotation column names: ['ID', 'ENTREZ_GENE_ID', 'Description', 'SPOT_ID']

First 5 rows of gene annotation data:
         ID ENTREZ_GENE_ID                                        Description  \
0      1_at              1                             alpha-1-B glycoprotein   
1     10_at             10  N-acetyltransferase 2 (arylamine N-acetyltrans...   
2    100_at            100                                adenosine deaminase   
3   1000_at           1000          cadherin 2, type 1, N-cadherin (neuronal)   
4  10000_at          10000      v-akt murine thymoma viral oncogene homolog 3   

   SPOT_ID  
0      1.0  
1     10.0  
2    100.0  
3   1000.0  
4  10000.0  

After normalization - First 10 gene symbols:
Index(['A1BG', 'A2M', 'A4GALT', 'AAA1', 'AAR2', 'AATK', 'ABCC11', 'ABCC5',
       'ABCD1', 'ABCE1'],
      dtype='object', name='Gene')

Total number of genes after mapping: 1884


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
# Note: We already normalized the gene data in the previous step
normalized_gene_data = gene_data.copy()
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Gene data saved to {out_gene_data_file}")

# Re-extract clinical features since we need it for linking
selected_clinical_df = geo_select_clinical_features(
    clinical_df=clinical_data,
    trait=trait,
    trait_row=trait_row,
    convert_trait=convert_trait,
    age_row=age_row,
    convert_age=convert_age,
    gender_row=gender_row,
    convert_gender=convert_gender
)

# 2. Link the clinical and genetic data with the 'geo_link_clinical_genetic_data' function from the library.
linked_data = geo_link_clinical_genetic_data(selected_clinical_df, normalized_gene_data)
print(f"Linked data shape before handling missing values: {linked_data.shape}")

# 3. Handle missing values in the linked data
linked_data = handle_missing_values(linked_data, trait)
print(f"Linked data shape after handling missing values: {linked_data.shape}")

# 4. Determine whether the trait and some demographic features are severely biased, and remove biased features.
is_trait_biased, unbiased_linked_data = judge_and_remove_biased_features(linked_data, trait)

# 5. Conduct quality check and save the cohort information.
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=True, 
    is_trait_available=True, 
    is_biased=is_trait_biased, 
    df=unbiased_linked_data,
    note="Dataset contains gene expression from endothelial cells derived from circulating progenitors of RA patients"
)

# 6. If the linked data is usable, save it as a CSV file to 'out_data_file'.
if is_usable:
    print(f"Data is usable. Saving to {out_data_file}")
    unbiased_linked_data.to_csv(out_data_file)
else:
    print("Data is not usable. Not saving linked data file.")

Gene data saved to ../../output/preprocess/Rheumatoid_Arthritis/gene_data/GSE121894.csv
Linked data shape before handling missing values: (58, 1885)


Linked data shape after handling missing values: (58, 1885)
For the feature 'Rheumatoid_Arthritis', the least common label is '0.0' with 22 occurrences. This represents 37.93% of the dataset.
The distribution of the feature 'Rheumatoid_Arthritis' in this dataset is fine.

A new JSON file was created at: ../../output/preprocess/Rheumatoid_Arthritis/cohort_info.json
Data is usable. Saving to ../../output/preprocess/Rheumatoid_Arthritis/GSE121894.csv
